In [1]:
import ethnicolr as ec
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pypopulation
warnings.filterwarnings("ignore")
import random
import statistics
import statistics
from matplotlib.ticker import MaxNLocator
import os 
os.chdir("/Users/alexesmerritt/Library/CloudStorage/GoogleDrive-akm147@georgetown.edu/My Drive/citation_bias/")


In [ ]:
df0 = pd.read_csv("dworkin_code_api/coreidd/df7_articledata_withgenders.csv")
df0[['FALN','FAFN']] = df0['first_auth'].str.split(', ', expand=True)
df0['first_auth']=df0['FAFN']+' '+df0['FALN']


df0[['LALN','LAFN']] = df0['last_auth'].str.split(', ', expand=True)
df0['last_auth']=df0['LAFN']+' '+df0['LALN']


In [ ]:
df_FArace=pd.read_csv('namsor_inputs/namsor_full-name-us-race_highlycited_processed_namsor_input.csv')
df_FArace=df_FArace[['name','raceEthnicity','probabilityCalibrated']]

df_LArace=pd.read_csv('namsor_inputs/namsor_full-name-us-race_highlycited_processed_namsor_input (1).csv')
df_LArace=df_LArace[['name','raceEthnicity','probabilityCalibrated']]


df_namsor_race= pd.concat([df_FArace, df_LArace])
df_namsor_race = df_namsor_race.sort_values('probabilityCalibrated',ascending=False).drop_duplicates(subset=['name'])

df_new_namsor_race=pd.read_csv("dworkin_code_api/coreidd/namsor_race_output_coreidd.csv")
df_new_namsor_race=df_new_namsor_race[['name','raceEthnicity','probabilityCalibrated']]

df_namsor_race=pd.concat([df_namsor_race,df_new_namsor_race])
df_namsor_race = df_namsor_race.sort_values('probabilityCalibrated',ascending=False).drop_duplicates(subset=['name'])



namsor_race=dict(zip(df_namsor_race.name.str.lower(), df_namsor_race.raceEthnicity))
namsor_race_prob=dict(zip(df_namsor_race.name.str.lower(),df_namsor_race.probabilityCalibrated))


df0['first_auth']=df0['first_auth'].str.lower()
df0['FA_race'] = df0['first_auth'].map(namsor_race)
df0['FA_race_prob'] = df0['first_auth'].map(namsor_race_prob)

df0['last_auth']=df0['last_auth'].str.lower()
df0['LA_race'] = df0['last_auth'].map(namsor_race)
df0['LA_race_prob'] = df0['last_auth'].map(namsor_race_prob)

In [ ]:
df_FAgender=pd.read_csv('namsor_inputs/namsor_genderize-full-name_highlycited_processed_namsor_input.csv')
df_FAgender=df_FAgender[['name','likelyGender','probabilityCalibrated']]


df_LAgender=pd.read_csv('namsor_inputs/namsor_genderize-full-name_highlycited_processed_namsor_input (1).csv')
df_LAgender=df_LAgender[['name','likelyGender','probabilityCalibrated']]


df_namsor_gender= pd.concat([df_FAgender, df_LAgender])
df_namsor_gender = df_namsor_gender.sort_values('probabilityCalibrated',ascending=False).drop_duplicates(subset=['name'])


df_new_namsor_gender=pd.read_csv("dworkin_code_api/coreidd/namsor_gender_output_coreidd.csv")
df_new_namsor_gender=df_new_namsor_gender[['name','likelyGender','probabilityCalibrated']]
df_namsor_gender=pd.concat([df_namsor_gender,df_new_namsor_gender])
df_namsor_gender = df_namsor_gender.sort_values('probabilityCalibrated',ascending=False).drop_duplicates(subset=['name'])

namsor_gender=dict(zip(df_namsor_gender.name.str.lower(), df_namsor_gender.likelyGender))
namsor_gender_prob=dict(zip(df_namsor_gender.name.str.lower(),df_namsor_gender.probabilityCalibrated))


df0['first_auth']=df0['first_auth'].str.lower()
df0['FA_gender'] = df0['first_auth'].map(namsor_gender)
df0['FA_gender_prob'] = df0['first_auth'].map(namsor_gender_prob)

df0['last_auth']=df0['last_auth'].str.lower()
df0['LA_gender'] = df0['last_auth'].map(namsor_gender)
df0['LA_gender_prob'] = df0['last_auth'].map(namsor_gender_prob)

In [ ]:
df0.to_csv("dworkin_code_api/coreidd/namsor_completed_df7.5.csv")

In [11]:
df=pd.read_csv("dworkin_code_api/coreidd/namsor_completed_df7.5.csv")

160228

In [3]:
# Here you will see a series of runs of the same program, ethnicolr times out and had problems with large datasets.
### To work around this we ran the datasets and then find which names it wasnt able to produce a prediction for.
### Getting this took us 4 rounds of ethnicolr.

#Creating Dataframes
df_LA=df[['UT','LALN','LAFN']]
df_FA=df[['UT','FALN', 'FAFN']]

#Ethnicolr 1 FA
results_FA = ec.pred_fl_reg_name(df_FA, 'FALN', 'FAFN')
results_FA=results_FA.rename(columns={'asian':'FA_asian', 'hispanic':'FA_hispanic', 'nh_black':'FA_black',
       'nh_white':'FA_white', 'race':'FA_ethni_race'})
#Ethnicolr 1 LA
results_LA = ec.pred_fl_reg_name(df_LA, 'LALN', 'LAFN')
results_LA=results_LA.rename(columns={'asian':'LA_asian', 'hispanic':'LA_hispanic', 'nh_black':'LA_black',
       'nh_white':'LA_white', 'race':'LA_ethni_race'})


In [4]:
#Ethnicolr 2

#Finding Nan Papers
UT_FA_nan=main_list = list(set(df['UT']) - set(results_FA['UT']))

UT_LA_nan=main_list = list(set(df['UT']) - set(results_LA['UT']))

#Making Datasets

df_FA_nan=df.loc[df['UT'].isin(UT_FA_nan)]

df_LA_nan=df.loc[df['UT'].isin(UT_LA_nan)]

df_LA_nan=df_LA_nan[['UT','LALN','LAFN']]
df_FA_nan=df_FA_nan[['UT','FALN', 'FAFN']]

#Ethnicolr 2 FA
results_FA_nan = ec.pred_fl_reg_name(df_FA_nan, 'FALN', 'FAFN')
results_FA_nan=results_FA_nan.rename(columns={'asian':'FA_asian', 'hispanic':'FA_hispanic', 'nh_black':'FA_black',
       'nh_white':'FA_white', 'race':'FA_ethni_race'})
#Ethnicolr 2 LA
results_LA_nan = ec.pred_fl_reg_name(df_LA_nan, 'LALN', 'LAFN')
results_LA_nan=results_LA_nan.rename(columns={'asian':'LA_asian', 'hispanic':'LA_hispanic', 'nh_black':'LA_black',
       'nh_white':'LA_white', 'race':'LA_ethni_race'})

In [5]:
#Merging Ethnicolr 1 and 2
results_FA=pd.concat([results_FA,results_FA_nan],ignore_index=True)
results_LA=pd.concat([results_LA,results_LA_nan],ignore_index=True)


In [6]:
#Ethnicolr 3
UT_FA_nan=main_list = list(set(df['UT']) - set(results_FA['UT']))

UT_LA_nan=main_list = list(set(df['UT']) - set(results_LA['UT']))

df_FA_nan=df.loc[df['UT'].isin(UT_FA_nan)]

df_LA_nan=df.loc[df['UT'].isin(UT_LA_nan)]
#Make Data Frames
df_LA_nan=df_LA_nan[['UT','LALN','LAFN']]
df_FA_nan=df_FA_nan[['UT','FALN', 'FAFN']]


#Ethnicolr 3 FA
results_FA_nan = ec.pred_fl_reg_name(df_FA_nan, 'FALN', 'FAFN')
results_FA_nan=results_FA_nan.rename(columns={'asian':'FA_asian', 'hispanic':'FA_hispanic', 'nh_black':'FA_black',
       'nh_white':'FA_white', 'race':'FA_ethni_race'})

#Ethniclor 3 LA
results_LA_nan = ec.pred_fl_reg_name(df_LA_nan, 'LALN', 'LAFN')
results_LA_nan=results_LA_nan.rename(columns={'asian':'LA_asian', 'hispanic':'LA_hispanic', 'nh_black':'LA_black',
       'nh_white':'LA_white', 'race':'LA_ethni_race'})

In [7]:
#Merging Results from 1&2 with 3
results_FA=pd.concat([results_FA,results_FA_nan],ignore_index=True)
results_LA=pd.concat([results_LA,results_LA_nan],ignore_index=True)

In [8]:
#Ethnicolr 4
UT_FA_nan=main_list = list(set(df['UT']) - set(results_FA['UT']))

UT_LA_nan=main_list = list(set(df['UT']) - set(results_LA['UT']))

df_FA_nan=df.loc[df['UT'].isin(UT_FA_nan)]

df_LA_nan=df.loc[df['UT'].isin(UT_LA_nan)]
#Make Data Frames
df_LA_nan=df_LA_nan[['UT','LALN','LAFN']]
df_FA_nan=df_FA_nan[['UT','FALN', 'FAFN']]


#Ethnicolr 4 FA
results_FA_nan = ec.pred_fl_reg_name(df_FA_nan, 'FALN', 'FAFN')
results_FA_nan=results_FA_nan.rename(columns={'asian':'FA_asian', 'hispanic':'FA_hispanic', 'nh_black':'FA_black',
       'nh_white':'FA_white', 'race':'FA_ethni_race'})

#Ethniclor 4 LA
results_LA_nan = ec.pred_fl_reg_name(df_LA_nan, 'LALN', 'LAFN')
results_LA_nan=results_LA_nan.rename(columns={'asian':'LA_asian', 'hispanic':'LA_hispanic', 'nh_black':'LA_black',
       'nh_white':'LA_white', 'race':'LA_ethni_race'})

In [9]:
#Merging Results from 1,2,3 with 4
results_FA=pd.concat([results_FA,results_FA_nan],ignore_index=True)
results_LA=pd.concat([results_LA,results_LA_nan],ignore_index=True)



results=results_LA.merge(results_FA,on='UT',how='outer')

results['LA_ethni_prop']=''
results['LA_ethni_prop'].loc[results['LA_ethni_race']=='asian']=results['LA_asian'].loc[results['LA_ethni_race']=='asian']
results['LA_ethni_prop'].loc[results['LA_ethni_race']=='nh_white']=results['LA_white'].loc[results['LA_ethni_race']=='nh_white']
results['LA_ethni_prop'].loc[results['LA_ethni_race']=='hispanic']=results['LA_hispanic'].loc[results['LA_ethni_race']=='hispanic']
results['LA_ethni_prop'].loc[results['LA_ethni_race']=='nh_black']=results['LA_black'].loc[results['LA_ethni_race']=='nh_black']

results['FA_ethni_prop']=''
results['FA_ethni_prop'].loc[results['FA_ethni_race']=='asian']=results['FA_asian'].loc[results['FA_ethni_race']=='asian']
results['FA_ethni_prop'].loc[results['FA_ethni_race']=='nh_white']=results['FA_white'].loc[results['FA_ethni_race']=='nh_white']
results['FA_ethni_prop'].loc[results['FA_ethni_race']=='hispanic']=results['FA_hispanic'].loc[results['FA_ethni_race']=='hispanic']
results['FA_ethni_prop'].loc[results['FA_ethni_race']=='nh_black']=results['FA_black'].loc[results['FA_ethni_race']=='nh_black']




print(results['FA_ethni_race'].unique())

results=results[['UT','FA_ethni_race','LA_ethni_race','FA_ethni_prop','LA_ethni_prop']]
df_comp=df.merge(results,on='UT',how='outer')
df_comp=df_comp.replace({'nh_white':'W_NL', 'asian':'A', 'hispanic':"HL", 'nh_black':'B_NL'})

df_comp=df_comp.rename(columns={'LA_race_prob':'LA_prob_race','FA_race_prob':'FA_prob_race'})
df_comp=df_comp.rename(columns={'LA_gender_prob':'LA_prob_gender','FA_gender_prob':'FA_prob_gender'})
print(df_comp.columns)
df_comp.to_csv("Alexes' Section/Prethresholding_HC.csv")

# Outputing a file that has results for the papers wherever possible and 

['hispanic' 'nh_white' 'nh_black' 'asian' nan]
Index(['Unnamed: 0', 'AF', 'SO', 'DI', 'PY', 'PD', 'DT', 'WC', 'UT', 'TC',
       'PM', 'TI', 'done', 'fa_fname', 'la_fname', 'prob.m.fa', 'prob.w.fa',
       'prob.m.la', 'prob.w.la', 'first_auth', 'last_auth', 'FALN', 'FAFN',
       'LALN', 'LAFN', 'FA_race', 'FA_prob_race', 'LA_race', 'LA_prob_race',
       'FA_gender', 'FA_prob_gender', 'LA_gender', 'LA_prob_gender',
       'FA_ethni_race', 'LA_ethni_race', 'FA_ethni_prop', 'LA_ethni_prop'],
      dtype='object')


NameError: name 'df_' is not defined

In [10]:
df_comp=pd.read_csv("Alexes' Section/Prethresholding_HC.csv")
df_UT=df_comp[['UT']]
print(len(df_comp.loc[df_comp['UT'].isna()]))

df_UT['UT']=df_UT['UT'].str.replace('WOS', 'UT', regex=True)
print(len(df_UT.loc[df_UT['UT'].isna()]))
df_UT['UT'].to_csv("Alexes' Section/HC_UT.txt",index=False,header=False)

0
0


# Books

In [11]:
#############REPEATING ALL THE PREVIOUS STEPS FOR OUR DEFINITION OF THE FIELD BASED ON BOOKS.
df0 = pd.read_csv("/dworkin_code_api/books/df7_articledata_withgenders.csv")

df_FAgender=pd.read_csv('/namsor_inputs/namsor_genderize-full-name_books_processed_namsor_input.csv')
df_FAgender=df_FAgender.rename(columns={'likelyGender':'FA_gender', 'probabilityCalibrated':'FA_prob_gender'})
df_FAgender=df_FAgender[['UT','FA_gender', 'FA_prob_gender']]

df_LAgender=pd.read_csv('/namsor_inputs/namsor_genderize-full-name_books_processed_namsor_input (1).csv')
df_LAgender=df_LAgender.rename(columns={'likelyGender':'LA_gender', 'probabilityCalibrated':'LA_prob_gender'})
df_LAgender=df_LAgender[['UT','LA_gender', 'LA_prob_gender']]

df0=df0.merge(df_FAgender,on='UT',how='outer')
df0=df0.merge(df_LAgender,on='UT',how='outer')

df_FArace=pd.read_csv('/namsor_inputs/namsor_full-name-us-race_books_processed_namsor_input.csv')
df_FArace=df_FArace.rename(columns={'raceEthnicityAlt':'FA_race2', 'raceEthnicity':'FA_race',
                                    'probabilityCalibrated':'FA_prob_race','probabilityAltCalibrated':'FA_prob2'})
df_FArace=df_FArace[['UT','FA_race2', 'FA_race', 'FA_prob_race', 'FA_prob2']]

df_LArace=pd.read_csv('/namsor_inputs/namsor_full-name-us-race_books_processed_namsor_input (1).csv')
df_LArace=df_LArace.rename(columns={'raceEthnicityAlt':'LA_race2', 'raceEthnicity':'LA_race',
                                    'probabilityCalibrated':'LA_prob_race','probabilityAltCalibrated':'LA_prob2'})
df_LArace=df_LArace[['UT','LA_race2', 'LA_race', 'LA_prob_race', 'LA_prob2']]

df0=df0.merge(df_FArace,on='UT',how='outer')
df0=df0.merge(df_LArace,on='UT',how='outer')



In [12]:
df=df0

df['first_auth'] = df['first_auth'].astype(str)
df['last_auth'] = df['last_auth'].astype(str)
FAl=list(df['first_auth'])
LAl=list(df['last_auth'])

df['FAFN']=''
df['FALN']=''
df['LAFN']=''
df['LALN']=''
FAFN=list(df['FAFN'])
FALN=list(df['FALN'])
LAFN=list(df['LAFN'])
LALN=list(df['LALN'])




for i in range(0,len(df)): 
    if ', ' in LAl[i]:
        LA=LAl[i].split(', ')
        LAl[i]=LA[1]+' '+LA[0]
        LAFN[i]=LA[1]
        LALN[i]=LA[0]
    if ', ' in FAl[i]:
        FA=FAl[i].split(', ')
        FAl[i]=FA[1]+' '+LA[0]
        FAFN[i]=FA[1]
        FALN[i]=FA[0]

df['FAFN']=FAFN
df['FALN']=FALN
df['LAFN']=LAFN
df['LALN']=LALN
        
df['FAFN']=df['FAFN'].str.title()
df['FALN']=df['FALN'].str.title()
df['LAFN']=df['LAFN'].str.title()
df['LALN']=df['LALN'].str.title()
print(df['first_auth'].unique())

['Kabir, M. Humayun' 'Elsadany, A. A.' 'Igoe, Morganne' ...
 'MITSUYA, Hiroaki' 'NEEQUAYE, AR' 'BELSEY, MA']


In [ ]:
#First Run of Ethnicolr

#Creating Dataframes
df_LA=df[['UT','LALN','LAFN']]
df_FA=df[['UT','FALN', 'FAFN']]

#Ethnicolr 1 FA
results_FA = ec.pred_fl_reg_name(df_FA, 'FALN', 'FAFN')
results_FA=results_FA.rename(columns={'asian':'FA_asian', 'hispanic':'FA_hispanic', 'nh_black':'FA_black',
       'nh_white':'FA_white', 'race':'FA_ethni_race'})
#Ethnicolr 1 LA
results_LA = ec.pred_fl_reg_name(df_LA, 'LALN', 'LAFN')
results_LA=results_LA.rename(columns={'asian':'LA_asian', 'hispanic':'LA_hispanic', 'nh_black':'LA_black',
       'nh_white':'LA_white', 'race':'LA_ethni_race'})


In [ ]:
#Ethnicolr 2

#Finding Nan Papers
UT_FA_nan=main_list = list(set(df['UT']) - set(results_FA['UT']))

UT_LA_nan=main_list = list(set(df['UT']) - set(results_LA['UT']))

#Making Datasets

df_FA_nan=df.loc[df['UT'].isin(UT_FA_nan)]

df_LA_nan=df.loc[df['UT'].isin(UT_LA_nan)]

df_LA_nan=df_LA_nan[['UT','LALN','LAFN']]
df_FA_nan=df_FA_nan[['UT','FALN', 'FAFN']]

#Ethnicolr 2 FA
results_FA_nan = ec.pred_fl_reg_name(df_FA_nan, 'FALN', 'FAFN')
results_FA_nan=results_FA_nan.rename(columns={'asian':'FA_asian', 'hispanic':'FA_hispanic', 'nh_black':'FA_black',
       'nh_white':'FA_white', 'race':'FA_ethni_race'})
#Ethnicolr 2 LA
results_LA_nan = ec.pred_fl_reg_name(df_LA_nan, 'LALN', 'LAFN')
results_LA_nan=results_LA_nan.rename(columns={'asian':'LA_asian', 'hispanic':'LA_hispanic', 'nh_black':'LA_black',
       'nh_white':'LA_white', 'race':'LA_ethni_race'})

In [ ]:
#Merging Ethnicolr 1 and 2
results_FA=pd.concat([results_FA,results_FA_nan],ignore_index=True)
results_LA=pd.concat([results_LA,results_LA_nan],ignore_index=True)


In [ ]:
#Ethnicolr 3
UT_FA_nan=main_list = list(set(df['UT']) - set(results_FA['UT']))

UT_LA_nan=main_list = list(set(df['UT']) - set(results_LA['UT']))

df_FA_nan=df0.loc[df0['UT'].isin(UT_FA_nan)]

df_LA_nan=df0.loc[df0['UT'].isin(UT_LA_nan)]
#Make Data Frames
df_LA_nan=df_LA_nan[['UT','LALN','LAFN']]
df_FA_nan=df_FA_nan[['UT','FALN', 'FAFN']]


#Ethnicolr 3 FA
results_FA_nan = ec.pred_fl_reg_name(df_FA_nan, 'FALN', 'FAFN')
results_FA_nan=results_FA_nan.rename(columns={'asian':'FA_asian', 'hispanic':'FA_hispanic', 'nh_black':'FA_black',
       'nh_white':'FA_white', 'race':'FA_ethni_race'})

#Ethniclor 3 LA
results_LA_nan = ec.pred_fl_reg_name(df_LA_nan, 'LALN', 'LAFN')
results_LA_nan=results_LA_nan.rename(columns={'asian':'LA_asian', 'hispanic':'LA_hispanic', 'nh_black':'LA_black',
       'nh_white':'LA_white', 'race':'LA_ethni_race'})

In [ ]:
#Merging Results from 1&2 with 3
results_FA=pd.concat([results_FA,results_FA_nan],ignore_index=True)
results_LA=pd.concat([results_LA,results_LA_nan],ignore_index=True)

In [ ]:
#Ethnicolr 4
UT_FA_nan=main_list = list(set(df['UT']) - set(results_FA['UT']))

UT_LA_nan=main_list = list(set(df['UT']) - set(results_LA['UT']))

df_FA_nan=df0.loc[df0['UT'].isin(UT_FA_nan)]

df_LA_nan=df0.loc[df0['UT'].isin(UT_LA_nan)]
#Make Data Frames
df_LA_nan=df_LA_nan[['UT','LALN','LAFN']]
df_FA_nan=df_FA_nan[['UT','FALN', 'FAFN']]


#Ethnicolr 4 FA
results_FA_nan = ec.pred_fl_reg_name(df_FA_nan, 'FALN', 'FAFN')
results_FA_nan=results_FA_nan.rename(columns={'asian':'FA_asian', 'hispanic':'FA_hispanic', 'nh_black':'FA_black',
       'nh_white':'FA_white', 'race':'FA_ethni_race'})

#Ethniclor 4 LA
results_LA_nan = ec.pred_fl_reg_name(df_LA_nan, 'LALN', 'LAFN')
results_LA_nan=results_LA_nan.rename(columns={'asian':'LA_asian', 'hispanic':'LA_hispanic', 'nh_black':'LA_black',
       'nh_white':'LA_white', 'race':'LA_ethni_race'})

In [ ]:
#Merging Results from 1,2,3 with 4
results_FA=pd.concat([results_FA,results_FA_nan],ignore_index=True)
results_LA=pd.concat([results_LA,results_LA_nan],ignore_index=True)



results=results_LA.merge(results_FA,on='UT',how='outer')

results['LA_ethni_prop']=''
results['LA_ethni_prop'].loc[results['LA_ethni_race']=='asian']=results['LA_asian'].loc[results['LA_ethni_race']=='asian']
results['LA_ethni_prop'].loc[results['LA_ethni_race']=='nh_white']=results['LA_white'].loc[results['LA_ethni_race']=='nh_white']
results['LA_ethni_prop'].loc[results['LA_ethni_race']=='hispanic']=results['LA_hispanic'].loc[results['LA_ethni_race']=='hispanic']
results['LA_ethni_prop'].loc[results['LA_ethni_race']=='nh_black']=results['LA_black'].loc[results['LA_ethni_race']=='nh_black']

results['FA_ethni_prop']=''
results['FA_ethni_prop'].loc[results['FA_ethni_race']=='asian']=results['FA_asian'].loc[results['FA_ethni_race']=='asian']
results['FA_ethni_prop'].loc[results['FA_ethni_race']=='nh_white']=results['FA_white'].loc[results['FA_ethni_race']=='nh_white']
results['FA_ethni_prop'].loc[results['FA_ethni_race']=='hispanic']=results['FA_hispanic'].loc[results['FA_ethni_race']=='hispanic']
results['FA_ethni_prop'].loc[results['FA_ethni_race']=='nh_black']=results['FA_black'].loc[results['FA_ethni_race']=='nh_black']




print(results['FA_ethni_race'].unique())

results=results[['UT','FA_ethni_race','LA_ethni_race','FA_ethni_prop','LA_ethni_prop']]
df_comp=df.merge(results,on='UT',how='outer')
df_comp=df_comp.replace({'nh_white':'W_NL', 'asian':'A', 'hispanic':"HL", 'nh_black':'B_NL'})
df_comp.to_csv("Alexes' Section/Prethresholding_books.csv")


In [32]:
df_comp=pd.read_csv("Alexes' Section/Prethresholding_books.csv")
df_UT=df_comp[['UT']]
df_UT['UT']=df_UT['UT'].str.replace('WOS', 'UT', regex=True)
df_UT['UT'].to_csv("Alexes' Section/books_UT.txt",index=False,header=False)

0
0


# Thresholding

In [40]:
# FUnction that will group and threshold different any of the datasets
"/Users/alexesmerritt/Library/CloudStorage/GoogleDrive-akm147@georgetown.edu/My Drive/citation_bias/"
def thresholding(threshold,dataset):
    df=pd.read_csv("Alexes' Section/Prethresholding_"+dataset+".csv")
    ########################### THRESHOLDING FOR RACE ###########################
    non_white =['A', 'HL', 'B_NL']
    ######## Grouping namsore Race outputs
    #creating column to save output
    df['FA_group']=''
    df['LA_group']=''
    ##Putting asian, hispanic and black authors in the nonwhite category 'N'
    df['FA_group'].loc[df['FA_race'].isin(non_white)] = "N"
    df['LA_group'].loc[df['LA_race'].isin(non_white)] = "N"

    ## Putting white authors as white in grouping columns
    df['FA_group'].loc[df['FA_race'] == 'W_NL'] = "W"
    df['LA_group'].loc[df['LA_race'] == 'W_NL'] = "W"
    
    
    ######## Repeating process for Ethnicolr results
    df['FA_group_e']=''
    df['LA_group_e']=''
    df['FA_group_e'].loc[df['FA_ethni_race'].isin(non_white)] = "N"
    df['LA_group_e'].loc[df['LA_ethni_race'].isin(non_white)] = "N"
    df['FA_group_e'].loc[df['FA_ethni_race'] == 'W_NL'] = "W"
    df['LA_group_e'].loc[df['LA_ethni_race'] == 'W_NL'] = "W"
    df['FA_ethni_prop']=pd.to_numeric(df['FA_ethni_prop'], errors='coerce')
    df['LA_ethni_prop']=pd.to_numeric(df['LA_ethni_prop'], errors='coerce')
    
    ####################### SIMPLIFYING NAMSOR GENDER RESULT #################
    df['LA_gender']=df['LA_gender'].replace({'female':'W','male':'M'})
    df['FA_gender']=df['FA_gender'].replace({'female':'W','male':'M'})

    ############################## THRESHOLDING ############################
    
    df['FA_group_e'].loc[df['FA_ethni_prop']<threshold] = 'U'
    df['FA_group_e'].loc[df['FA_ethni_prop'].isna()] = 'U'
    df['LA_group_e'].loc[df['LA_ethni_prop']<threshold] = 'U'
    df['LA_group_e'].loc[df['LA_ethni_prop'].isna()] = 'U'
    df['groups_e']=df['FA_group_e']+df['LA_group_e']

    df['FA_ethni_race'].loc[df['FA_ethni_prop']<threshold] = 'U'
    df['LA_ethni_race'].loc[df['LA_ethni_prop']<threshold] = 'U'
    df['FA_ethni_race'].loc[df['FA_ethni_prop'].isna()] = 'U'
    df['LA_ethni_race'].loc[df['LA_ethni_prop'].isna()] = 'U'

    df['FA_group'].loc[df['FA_prob_race']<threshold] = 'U'
    df['LA_group'].loc[df['LA_prob_race']<threshold] = 'U'
    df['FA_group'].loc[df['FA_prob_race'].isna()] = 'U'
    df['LA_group'].loc[df['LA_prob_race'].isna()] = 'U'
    df['groups']=df['FA_group']+df['LA_group']

    df['FA_race'].loc[df['FA_prob_race']<threshold] = 'U'
    df['LA_race'].loc[df['LA_prob_race']<threshold] = 'U'
    df['FA_race'].loc[df['FA_prob_race'].isna()] = 'U'
    df['LA_race'].loc[df['LA_prob_race'].isna()] = 'U'
    
    df['FA_gender'].loc[df['FA_prob_gender']<threshold] = 'U'
    df['FA_gender'].loc[df['FA_prob_gender'].isna()] = 'U'
    df['LA_gender'].loc[df['LA_prob_gender']<threshold] = 'U'
    df['LA_gender'].loc[df['LA_prob_gender'].isna()] = 'U'
    df['AG_namsor']=df['FA_gender']+df['LA_gender']
    
    
    df['FA_group_g']='U'
    df['FA_group_g'].loc[df['prob.w.fa']>=threshold]='W'
    df['FA_group_g'].loc[df['prob.m.fa']>=threshold]='M'

    df['LA_group_g']='U'
    df['LA_group_g'].loc[df['prob.w.la']>=threshold]='W'
    df['LA_group_g'].loc[df['prob.m.la']>=threshold]='M'
    df['AG']=df['FA_group_g']+df['LA_group_g']
    
    #### SETTING PAPERS AS GLOBAL NORTH BASED ON LAST AUTHOR AFFILIATION
   # definition of global north comes from:
    # https://unctadstat.unctad.org/EN/Classifications/DimCountries_All_Hierarchy.pdf
    westernworld=[ 'Bermuda','Canada','Greenland','USA','Cyprus','Israel','Australia',
                  'Christmas Island','Cocos Island','Heard Island and McDonald Island','New Zealand',
                  'Norfolk Island','United States Minor Outlying Islands','Aland Islands','Albania','Andorra',
                  'Austria','Belarus','Belgium','Bosnia and Herzegovina','Bulgaria','Croatia','Czechia',
                  'Czechoslovakia','Germany','Denmark','Estonia','Faroe Islands','Finland','France','Gibraltar',
                 'Greece','Guernsey','Holy See','Hungary','Holy See','Hungary','Iceland','Ireland','Isle of Man',
                 'Italy','Jersey','Kosovo','Latvia','Liechtenstein','Lithuania','Luxembourg','Malta','Monaco',
                  'Montenegro','Netherlands','North Macedonia','Norway','Poland','Portugal',
                  'Republic of Macedonia','Romania','Russia','San Marino','Serbia','Serbia and Montenegro',
                 'Slovakia','Slovenia','Spain','Svalbard and Jan Mayen Islands','Sweden','Switzerland','Ukraine',
                 'United Kingdom','Yugoslavia']
    westernworld=[x.lower() for x in westernworld]
    if dataset=='HC':
        df_country=pd.read_csv("Alexes' Section/coreidd_exporter/HC_countries.csv")
    elif dataset=='books':
        df_country=pd.read_csv("Alexes' Section/coreidd_exporter/books_countries.csv")
        
    df_country=df_country.rename(columns={'Accession Number (UT)':'UT'})
    df_country=df_country[['UT','Country','Researcher/Author SeqNo (position)']]
    df_country=df_country.sort_values(by=['UT','Researcher/Author SeqNo (position)']).drop_duplicates(subset=['UT'],keep='last')
    df_country=df_country[['UT','Country']]
    df=pd.merge(df,df_country,how='left',on='UT')
    
    df['Country'] = df['Country'].str.lower()
    ## Editing countries to correct names or update to new names
    df['Country']=df['Country'].replace({'czech republic':'czechia','new caledonia':'france',
                                         'england':'United Kingdom (Great Britain)',
                                         'north ireland':'United Kingdom (Great Britain)',
                                         'wales':'United Kingdom (Great Britain)',
                                         'scotland':'United Kingdom (Great Britain)',
                                        'french guiana':'France','u arab emirates':'UAE',
                                        'dominican rep':'Dominican Republic',
                                         'dem rep congo':'Democratic Republic of Congress',
                                        'rep congo':'Republic of Congo','fed rep ger':'Germany',
                                        'west germany':'Germany','austl.':'Australia',
                                        'serbia monteneg':'Serbia and Montenegro','ussr':'Russia',
                                         'cent afr republ':'Central African Republic',
                                        'deutsch dem rep':'Germany','ger dem rep':'Germany',
                                        'ukssr':'Ukraine','east germany':'Germany','united arab rep':'UAE',
                                        'trin + tobago':'Trinidad and Tobago','trinidad tobago':'Trinidad and Tobago'})
    
    df['Country'] = df['Country'].str.lower()
    df['global_north']=0
    df['global_north'].loc[df['Country'].isin(westernworld)]=1

    if dataset=='HC':
        dataset='highlycited'
        df.to_csv("dworkin_code_api/coreidd/df8_"+dataset+"_"+str(threshold)+'.csv')
    else: 
        df.to_csv("dworkin_code_api/books/"+dataset+"_"+str(threshold)+'.csv')

thresholding(.7,'HC')
thresholding(.6,'HC')
thresholding(.8,'HC')
thresholding(.7,'books')
